In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pathlib
from dateutil.relativedelta import relativedelta
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import sys
sys.path.append('../..')

from epymetheus import (
    Trade, 
    History,
    TradeStrategy, 
    Universe,
)

import seaborn
seaborn.set_style('ticks')

In [2]:
df = pd.read_csv('US Equity.csv', index_col=0, parse_dates=True)
universe = Universe(df, name='US Equity')
universe.data

,AAPL,MSFT,AMZN,BRK-A,JPM,JNJ,WMT,BAC,PG,XOM
2000-01-01,3.194901,37.556007,76.125000,56100.0,28.443703,27.450361,47.997063,12.477510,29.387398,22.486614
2000-01-02,3.194901,37.556007,76.125000,56100.0,28.443703,27.450361,47.997063,12.477510,29.387398,22.486614
2000-01-03,3.478462,37.495686,89.375000,54800.0,26.681709,27.137581,46.391357,12.042428,28.750359,21.858601
2000-01-04,3.185191,36.229057,81.937500,52000.0,26.170267,26.144068,44.655510,11.327658,28.197153,21.439911
2000-01-05,3.231803,36.611080,69.750000,53200.0,26.008717,26.420055,43.744167,11.451962,27.660698,22.608728
...,...,...,...,...,...,...,...,...,...,...
2019-12-27,289.799988,158.960007,1869.800049,338920.0,138.252441,145.750000,119.589996,35.349998,126.089996,69.889999
2019-12-28,289.799988,158.960007,1869.800049,338920.0,138.252441,145.750000,119.589996,35.349998,126.089996,69.889999
2019-12-29,289.799988,158.960007,1869.800049,338920.0,138.252441,145.750000,119.589996,35.349998,126.089996,69.889999
2019-12-30,291.519989,157.589996,1846.890015,338750.0,137.745697,145.300003,119.400002,35.150002,124.470001,69.480003


In [3]:
class SimpleMeanReversion(TradeStrategy):
    """
    A simple Mean Reversion strategy that buys stocks for a month with
    the lowest percentile one month returns and sells the highest percentile returns.

    Parameters
    ----------
    - percentile : float
        The threshold to buy or sell.
        E.g. If 0.1, buy/sell stocks with returns of lowest/highest 10%.
    """
    # XXX changed to trend follow

    def logic(self, universe, percentile):

        train_period = relativedelta(months=1)
        trade_period = relativedelta(months=1)

        def trade_bds(universe, train_period, trade_period):
            """Yield begin dates of trade periods."""
            d = universe.bars[0] + train_period
            while d + trade_period <= universe.bars[-1]:
                yield d
                d += trade_period

        def tot_return(asset, begin_date, end_date):
            """Return total return of asset from begin_date to end_date."""
            u = universe.data
            b, e = u.at[begin_date, asset], u.at[end_date, asset]
            return e / b - 1

        num_buysell = int(universe.n_assets * percentile)

        for trade_bd in trade_bds(universe, train_period, trade_period):
            train_bd = trade_bd - train_period
            trade_ed = trade_bd + trade_period - relativedelta(days=1)
            train_ed = train_bd + train_period - relativedelta(days=1)

            key = lambda asset: tot_return(asset, train_bd, train_ed)
            sell = sorted(universe.assets, key=key)[:num_buysell]
            buy = sorted(universe.assets, key=key)[-num_buysell:]
            
            for asset in sell:
                lot = 1.0 / universe.data.at[trade_bd, asset]
                yield Trade(asset=asset, lot=lot, open_date=trade_bd, close_date=trade_ed)
            
            for asset in buy:
                lot = -1.0 / universe.data.at[trade_bd, asset]
                yield Trade(asset=asset, lot=lot, open_date=trade_bd, close_date=trade_ed)

In [21]:
strategy = SimpleMeanReversion(percentile=0.1)
strategy.context()

strategy.run(universe, verbose=True)

Evaluating wealth ...
Done.
Runtime : 0.7sec


In [17]:
pd.DataFrame(strategy.history).head()

,assets,lots,open_dates,close_dates,durations,open_prices,gains
0,WMT,0.024540,2000-02-01,2000-02-29,28 days,40.7498,-0.169329
1,JPM,-0.032337,2000-02-01,2000-02-29,28 days,30.9243,0.0492536
2,WMT,0.029392,2000-03-01,2000-03-31,30 days,34.0233,0.154512
3,AAPL,-0.246946,2000-03-01,2000-03-31,30 days,4.04947,-0.0422058
4,PG,0.065336,2000-04-01,2000-04-30,29 days,15.3055,0.0627201


In [20]:
pd.DataFrame(strategy.wealth).tail()

,wealth
7300,-1.33732
7301,-1.33732
7302,-1.33732
7303,-1.33732
7304,-1.33732


In [18]:
pd.DataFrame(strategy.transaction).head()

,AAPL,MSFT,AMZN,BRK-A,JPM,JNJ,WMT,BAC,PG,XOM
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
